In [ ]:
import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-90;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems
periodsOnly=true; 
paths=[]; #unless overridden we will convert this to a list of straight paths

#load("~/git-projects/periods/integrate/data/current.sage")
load("~/git-projects/period-suite/current.sage")
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "";
    print "Integrating system number", j;
    print "";
    %time transitionMatricies = [deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true) for i in [1..noOfdeqs]]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(300), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:geometricGenus]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods


Integrating systems 1 through 3

Integrating system number 1

CPU times: user 2.42 s, sys: 198 ms, total: 2.62 s
Wall time: 4.6 s
Maximal error in transition matricies:
[1.3369889e-92, 1.5598216e-112, 1.5598216e-112, 3.6682862e-111, 1.5598216e-112, 3.6682862e-111, 9.9345915e-111, 7.1763620e-93, 1.1640620e-111, 7.1763620e-93, 1.1640620e-111, 2.6131978e-91, 1.1640620e-111, 2.6131978e-91, 1.3369889e-92, 5.6788872e-92, 1.0952692e-111, 5.6788872e-92, 1.0952692e-111, 1.0952692e-111, 9.9345915e-111]

Integrating system number 2



In [11]:
# Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 40
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 65
##
##

periodVector=periods
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]


sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]
j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if norms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms[j], testRelations[j]
    j=j+1


Picard rank appears to be 12 

Check to see if this makes sense:

01 || ---hyperplane section---
02 || 1.41421356237310 (0, 0)
03 || 1.41421356237310 (0, 0)
04 || 1.41421356237310 (0, 0)
05 || 1.41421356237310 (0, 0)
06 || 1.41421356237310 (0, 0)
07 || 1.41421356237310 (0, 0)
08 || 1.41421356237310 (0, 0)
09 || 1.41421356237310 (0, 0)
10 || 1.41421356237310 (0, 0)
11 || 1.41421356237310 (0, 0)
12 || 3.00000000000000 (-1, 0)
13 || 4.48756303648148e29 (43106058976972578915698991241, 446681192511418411437281657786)
14 || 8.81044165546021e29 (776128196175714374339425891791, 416969836731287631016139212513)
15 || 1.87556168770473e30 (1783996987501353175744882234827, 578866472489034490210606069192)
16 || 2.09430162768391e30 (-816795650971381230591288876269, 1928456422186856665081014380284)
17 || 1.47657163386534e35 (145644051021262107170629604538899114, 24299141990484244099741844766288658)
18 || 3.98818455145976e34 (39679109239138088465783033853637511, 4016203635123421170332022734894440)
19 

In [47]:
reducedLattice.row(16)

(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)